This script provides input files and steps for visulizing interaction network of ASVs (figure 6)

In [33]:
# import packages
library(dplyr)
library(tidyr)


In [55]:
# make a input table for calculation correlation matrix(select only lb plant samples and rmove ASVs with less than 50 reads)

asv <- read.table("../data/ASV_Table.csv" ,sep = "," , header = T )
rownames(asv) <- asv$X
taxa = read.table("../data/TaxaTable.csv" ,sep = "," , header = T ,stringsAsFactors = FALSE)
taxa$Genus <- taxa$Genus %>% replace_na("unknown")
taxa$Genus <- gsub("Allorhizobium-Neorhizobium-Pararhizobium-Rhizobium" , 'ANPR' ,taxa$Genus )
asvSam = merge(x <-taxa[c(1,7)], y = t(asv) ,  by.x = "X" , by.y = "row.names")
rownames(asvSam) <- paste(asvSam$X,asvSam$Genus , sep="_")
asvSam1 <- asvSam[-c(1,2)]

sample <- data.frame(asv$X)
sample1 <- sample %>% separate(asv.X, c("plant","soil","phenotype","replicate") , remove = F)
sample1$soil_phenotype_plant <-  paste(sample1$soil,sample1$phenotype,sample1$plant,sep = "_")

#table(sample1$soil_phenotype_plant)
samasv = merge(x <- sample1 , y = t(asvSam1) , by.x = "asv.X" , by.y = "row.names")
rownames(samasv) <- samasv$asv.X

#all allBsoil2
lb <- subset(samasv ,soil_phenotype_plant == "2_Healthy_Lb"| soil_phenotype_plant == "2_Starved_Lb" )
lb <- lb[-c(1:6)]
lb <- as.data.frame(t(lb))
lb1 <- apply(lb[,], 2,function(x) as.numeric(as.character(x)))
rownames(lb1) <- rownames(lb)
n <- data.frame(rowSums(lb1!=0))
n$sum <- rowSums(lb1)
n$name <- rownames(n)
nH1 <- n
lb1 <- as.data.frame(lb1[which(rowSums(lb1)>=50),])
lb2 <- tibble::rownames_to_column(lb1, "#OTU ID")
write.table(lb2, "../data/data1.txr", quote = F , sep = "\t" , col.names = T , row.names = F)
head(lb2)


,#OTU ID,Lb_2_Healthy_A,Lb_2_Healthy_B,Lb_2_Healthy_C,Lb_2_Healthy_D,Lb_2_Healthy_E,Lb_2_Healthy_F,Lb_2_Healthy_G,Lb_2_Healthy_H,Lb_2_Healthy_I,⋯,Lb_2_Starved_D,Lb_2_Starved_E,Lb_2_Starved_F,Lb_2_Starved_G,Lb_2_Starved_H,Lb_2_Starved_I,Lb_2_Starved_J,Lb_2_Starved_K,Lb_2_Starved_L,Lb_2_Starved_M
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ASV1_Mesorhizobium,158682,96522,79175,185493,79161,118648,157116,159759,190633,⋯,260,91388,39483,1511,95205,33101,51779,27003,144,2271
2,ASV1032_unknown,0,0,0,0,0,0,0,42,0,⋯,0,0,0,0,0,19,0,0,0,0
3,ASV104_Mesorhizobium,0,0,385,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,ASV1049_unknown,0,0,0,0,0,0,0,0,0,⋯,8,0,22,11,0,0,0,0,0,0
5,ASV106_Mesorhizobium,0,0,0,0,92,0,0,0,0,⋯,57,0,269,195,0,127,97,93,0,23
6,ASV108_Mesorhizobium,0,0,347,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


# calculate correlation matrix 

FastSpar(github.com/scwatts/FastSpar) was used to calculate correlation matrix via command line.   

mkdir bootstrap_counts bootstrap_correlation
#### step1: Correlation inference
fastspar --otu_table data1.txt  --correlation median_correlation.tsv --covariance median_covariance.tsv > log1

#### step2: Calculation of exact p-values, first generate 1000 bootstrap counts
fastspar_bootstrap --otu_table data1.txt   --number 1000 --prefix bootstrap_counts/ > log2
#### step3: infer correlations for each bootstrap count (running in parallel):
parallel -j 12 fastspar --otu_table {} --correlation bootstrap_correlation/cor_{/} --covariance bootstrap_correlation/cov_{/} -i 50 ::: bootstrap_counts/*  > log3
#### step4: From these correlations, the p-values are then calculated:
fastspar_pvalues --otu_table data1.txt   --correlation median_correlation.tsv --prefix bootstrap_correlation/cor_  --permutations 1000 --outfile pvalues.tsv > log4

#### output files are in data folder : median_correlation.tsv and pvalues.tsv , remove "#" from the files before opening in R



#### Step5: Take the correlation of Pseudomonas ASVs with ANPR and Mesorisubiums from correlation matrix



In [48]:

Pvals <- read.table(file = "../data/pvalues.tsv", sep="\t", header=T, row.names=1 ) 
Pvals1 <- as.matrix(Pvals)
Pvals1[upper.tri(Pvals, diag=TRUE)]<- NA
Pvals2 <- as.data.frame(as.table(Pvals1))
colnames(Pvals2)<-c("Node1","Node2","Pvalue")
  
Cors <- read.table("../data/median_correlation.tsv", sep="\t", header=T, row.names=1) 
Cors1 <- as.matrix(Cors)
Cors1[upper.tri(Cors1, diag=TRUE)]<-NA
Cors2 <- as.data.frame(as.table(Cors1))
colnames(Cors2)<-c("Node1","Node2","Cor")
  
Edge_table <- cbind(Pvals2,Cors2$Cor, deparse.level=2)
Edge_table_final <- Edge_table[!is.na(Edge_table$Pvalue),]
colnames(Edge_table_final) <- c("Node1","Node2","Pvalue","Cor")
Edge_table_final <- subset(Edge_table_final,abs(Edge_table_final$Cor)>=0.2)
Edge_table_final$CorrB <- ifelse(Edge_table_final$Cor >=0, "Positive", "Negative")
Edge_table_final$CorRange <- ifelse(abs(Edge_table_final$Cor) >=0.4, "Strong", "Weak")
Edge_table_final_P0.01 <- Edge_table_final[which(Edge_table_final$Pvalue<=0.01),]
#table(Edge_table_final_P0.01$CorrB)
Edge_table_final_P0.01$CorrStrong = paste(Edge_table_final_P0.01$CorrB , Edge_table_final_P0.01$CorRange , sep = "_")
#write.table(Edge_table_final_P0.01, file=paste("Pval/0.01_",basename(corfilelist[index])), sep="\t", row.names=FALSE , quote = F)

# subset correlation of pseudomonas with ANPR andMesorhizobium
file <- Edge_table_final_P0.01
file$name = paste(file$Node1,file$Node2, sep = "_")
file <- file %>% separate(name, c("v1","n1","v2","n2") , remove = F)
file$name2 <- paste(file$n1,file$n2,sep = "_")
final <- subset(file, name2=="Pseudomonas_Pseudomonas" | name2 == "ANPR_ANPR" | name2=="Mesorhizobium_Mesorhizobium" | 
                  name2=="Pseudomonas_ANPR" | name2 =="ANPR_Pseudomonas" | name2=="Pseudomonas_Mesorhizobium" |
                  name2 =="Mesorhizobium_Pseudomonas" | name2=="ANPR_Mesorhizobium" | name2 =="Mesorhizobium_ANPR")


final<- subset(final,final$Pvalue < 0.01) 
final$CorRange <- ifelse(abs(final$Cor) >=0.4, "Strong", "Weak")

final1 <- subset(final, final$n1=="Pseudomonas" | final$n2=="Pseudomonas")
write.table(final1,"../data/InputEdgetable.csv", sep=",", row.names=FALSE , quote = F)

In [46]:
head(final1)

,Node1,Node2,Pvalue,Cor,CorrB,CorRange,CorrStrong,name,v1,n1,v2,n2,name2
,<fct>,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
834,ASV48_Pseudomonas,ASV106_Mesorhizobium,0.004,-0.3872,Negative,Weak,Negative_Weak,ASV48_Pseudomonas_ASV106_Mesorhizobium,ASV48,Pseudomonas,ASV106,Mesorhizobium,Pseudomonas_Mesorhizobium
846,ASV57_Pseudomonas,ASV106_Mesorhizobium,0.004,-0.4016,Negative,Strong,Negative_Strong,ASV57_Pseudomonas_ASV106_Mesorhizobium,ASV57,Pseudomonas,ASV106,Mesorhizobium,Pseudomonas_Mesorhizobium
866,ASV717_Pseudomonas,ASV106_Mesorhizobium,0.001,-0.3831,Negative,Weak,Negative_Weak,ASV717_Pseudomonas_ASV106_Mesorhizobium,ASV717,Pseudomonas,ASV106,Mesorhizobium,Pseudomonas_Mesorhizobium
1940,ASV717_Pseudomonas,ASV112_Mesorhizobium,0.005,0.3815,Positive,Weak,Positive_Weak,ASV717_Pseudomonas_ASV112_Mesorhizobium,ASV717,Pseudomonas,ASV112,Mesorhizobium,Pseudomonas_Mesorhizobium
5309,ASV48_Pseudomonas,ASV139_ANPR,0.007,-0.3291,Negative,Weak,Negative_Weak,ASV48_Pseudomonas_ASV139_ANPR,ASV48,Pseudomonas,ASV139,ANPR,Pseudomonas_ANPR
5321,ASV57_Pseudomonas,ASV139_ANPR,0.006,-0.3241,Negative,Weak,Negative_Weak,ASV57_Pseudomonas_ASV139_ANPR,ASV57,Pseudomonas,ASV139,ANPR,Pseudomonas_ANPR


In [54]:
f3PsudoANP <- with(final1, final1[ grepl( 'Pseudomonas',final1$name) & grepl( 'ANPR',final1$name) , ])
print("number of correlation between Pseudomonas and ANPR ASVs:")
table(f3PsudoANP$CorrB)


f3PsudoMeso <- with(final1, final1[ grepl( 'Pseudomonas',final1$name) & grepl( 'Mesorhizobium',final1$name) , ])
print("number of correlation between Pseudomonas and Mesorhizobium ASVs:")
table(f3PsudoMeso$CorrB)

[1] "number of correlation between Pseudomonas and ANPR ASVs:"



Negative 
      34 

[1] "number of correlation between Pseudomonas and Mesorhizobium ASVs:"



Negative Positive 
      21       28 

# Network visulization

#### InputEdgetable.csv table was open in cytoscape program for visulization of following network(figure 6)


![title](../fig6.png)